In [1]:
!rm -rf RLSnake
!git clone https://github.com/lifeskillsSI/rlsnake
!pip install pygame

Cloning into 'rlsnake'...
remote: Enumerating objects: 10212, done.
remote: Counting objects: 100% (10212/10212), done.
remote: Compressing objects: 100% (9782/9782), done.
remote: Total 10212 (delta 369), reused 10193 (delta 354), pack-reused 0
Receiving objects: 100% (10212/10212), 23.18 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (369/369), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 62.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mv ./rlsnake/* .

In [ ]:
from dataclasses import dataclass
import random
import time
from vectors import *
from consts import *
import numpy as np
from agent import Agent
from game import Game

def manual_control():
    game = Game(0)
    time_elapsed_since_last_action = 0
    clock = pygame.time.Clock()
    req_dir = UP

    running = True

    while running:
        events = pygame.event.get()
        for event in events:
            if event.type == pygame.KEYDOWN and event.key in KEY_TO_M.keys():
                if game.direction not in KEY_TO_M[event.key]:
                    req_dir = KEY_TO_M[event.key][0]
            elif event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
                running = False
        
        delta = clock.tick()
        time_elapsed_since_last_action += delta
        if time_elapsed_since_last_action > 100:
            game.direction = req_dir
            time_elapsed_since_last_action = 0

            (reward, game_alright, score) = game.next_step()
            game.display()
            print(reward)
            if not game_alright:
                break

def train(model_path = ""):
    running = True

    game = Game(0)
    agent = Agent()
    agent.load_model(model_path)
    print("Starting game no", agent.game_count)

    while running:
        current_state = game.get_qstate()
        (action, pred) = agent.get_action(current_state)

        game.direction = action
        (reward, game_alright, score) = game.next_step()
        #game.display()

        new_state = game.get_qstate()
        
        # train short memory
        agent.train_short_memory(current_state, pred, reward, new_state, game_alright)
        # append memory
        agent.memory.append((current_state, pred, reward, new_state, game_alright))

        if not game_alright or game.step_counter > 80*(game.score+3):
            game = Game(0)
            # train long memory
            agent.replay_mem(1000)
            print("Starting game no", agent.game_count)
            if agent.game_count % 10 == 0:
                agent.model.save("./content/drive/MyDrive/RLSnake/a" + str(agent.game_count))
        
        #events = pygame.event.get()
        #for event in events:
        #    if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
        #        running = False

    pass

    #pygame.init()
    #SCREEN = pygame.display.set_mode([W * SCALE, H * SCALE])
train('./saves/10')

    #pygame.quit()


Starting game no 0
[[0.592664  0.0263946 0.3809414]]
1/1 [==============================] - 0s 304ms/step - loss: 0.0103
[[0.46385357 0.43683216 0.09931432]]
1/1 [==============================] - 0s 4ms/step - loss: 4.3721e-04
[[0.5557402  0.3981326  0.04612725]]
1/1 [==============================] - 0s 4ms/step - loss: 0.0229
[[0.90924597 0.06547626 0.02527775]]
1/1 [==============================] - 0s 5ms/step - loss: 0.0475
[[0.59078616 0.02533042 0.38388348]]
1/1 [==============================] - 0s 6ms/step - loss: 0.0095
[[0.47016448 0.43056864 0.0992669 ]]
1/1 [==============================] - 0s 8ms/step - loss: 5.2375e-04
[[0.56911385 0.38450873 0.04637747]]
1/1 [==============================] - 0s 5ms/step - loss: 0.0209
[[0.91076446 0.06388462 0.02535087]]
1/1 [==============================] - 0s 7ms/step - loss: 0.0492
[[0.5837654  0.02428911 0.39194557]]
1/1 [==============================] - 0s 5ms/step - loss: 0.0081
[[0.4767577  0.42367065 0.0995717 ]]
1/1 [=====